In [1]:
import serial
import datetime
import time
import serial.tools.list_ports
import pandas as pd

In [2]:
studentRecord = pd.read_csv("studentRecords.csv")
visitHistory = dict({"studentID":[], "name":[],"major":[],"Sign-in Time":[],"Sign-out Time":[]})
roomStatus = dict({"Students In":[]})
historyDF = pd.DataFrame()
studentRecord.head()

,studentID,name,major,Sign-in Time,Sign-out Time,signingIn
0,801606786,Samuel VanDenburgh,MECE,0,0,True
1,801735307,Noah Clippinger,Computer Engineering,0,0,True
2,801675087,Donald Hau,Electrical Engineering,0,0,True


In [3]:
comlist = serial.tools.list_ports.comports() #finds barcode port and opens it
connected = []
ser = None
for element in comlist:
    connected.append(element.device)
    print(element.description)
    if ("barcode scanner" in element.description) or ("USB Serial Device" in element.description):
        ser = serial.Serial(element.device,timeout=5)
        print("Barcode Scanner connected at:", ser.name)

if ser == None:
    exit(0)

USB Serial Device (COM5)
Barcode Scanner connected at: COM5
USB Serial Device (COM6)
Barcode Scanner connected at: COM6


In [4]:
def storeVisit(student): #Keeps a list of all recorded scans
        visitHistory["studentID"].append(student["studentID"])
        visitHistory["name"].append(student["name"])
        visitHistory["major"].append(student["major"])
        visitHistory["Sign-in Time"].append(student["Sign-in Time"])
        visitHistory["Sign-out Time"].append(student["Sign-out Time"]) #store values into JSON

In [5]:
def signingIn(student): #Keeps a list of all recorded scans
        #edit students scan status and time
        student["Sign-in Time"] = datetime.datetime.now().time()
        student['signingIn'] = "False"
        print("Student Signed In!")

In [6]:
def signingOut(student): #Keeps a list of all recorded scans
        #edit students scan status and histoyr
        student["Sign-out Time"] = datetime.datetime.now().time()
        #record visit
        student['signingIn'] = "True"
        print("Student Signed Out :(")

In [7]:
def getStudent(line):
    #find student with matching ID
    for student in studentRecord:
        if student["studentID"] == line:
            return student
        else:
            print("Student Not Found")

In [8]:
lasttime = time.time()
last_id = None
try:
    while True: #checks for new input from barcode and updates dataframe
        line = ser.readline()
        if len(line) > 3:
            line = line.decode('utf-8',errors='ignore').rstrip('/r/n')
            try:
                line = int(line)
                print(line)
                if last_id != line:
                    student = studentRecord.loc[studentRecord['studentID'] == line]
                    print(student)
                    ser.flush()
                    print("\n Student Scanned:\n Name: ",student.name,"\n\n")
                    if str(student['signingIn']) == "True": #If student is signing in
                        signingIn(student)
                    else: #If student is signing out
                        signingOut(student)
                        storeVisit(student)
                    #If time is pass hours
                    #    for student in Students
                    last_id = line
                else: 
                    print("double scanned")
            except ValueError:
                print("not an int value, was:", line)
        if time.time()-lasttime > 5: 
            last_id = None
            lasttime = time.time()
except KeyboardInterrupt:
    ser.close()

801606786
   studentID                name major  Sign-in Time  Sign-out Time  signingIn
0  801606786  Samuel VanDenburgh  MECE             0              0       True

 Student Scanned:
 Name:  0    Samuel VanDenburgh
Name: name, dtype: object 


Student Signed Out :(


C:\Users\haudj\AppData\Local\Temp\ipykernel_17824\3876710155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student["Sign-out Time"] = datetime.datetime.now().time()
C:\Users\haudj\AppData\Local\Temp\ipykernel_17824\3876710155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student['signingIn'] = "True"


801675087
   studentID        name                   major  Sign-in Time  Sign-out Time   
2  801675087  Donald Hau  Electrical Engineering             0              0  \

   signingIn  
2       True  

 Student Scanned:
 Name:  2    Donald Hau
Name: name, dtype: object 


Student Signed Out :(


C:\Users\haudj\AppData\Local\Temp\ipykernel_17824\3876710155.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student["Sign-out Time"] = datetime.datetime.now().time()
C:\Users\haudj\AppData\Local\Temp\ipykernel_17824\3876710155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  student['signingIn'] = "True"
